In [2]:
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import datetime
from datetime import datetime
import numpy as np
import matplotlib as mpl
import re
from fastai.tabular import *

In [3]:
df_m = pd.read_csv("2007-2020.csv", header=0)
df_m.Place = df_m.Place.str.lower()
df_m.head() #(7680, 3)

,Date,Place,Ill
0,2010-12-16,dolnośląskie,1528
1,2010-12-08,dolnośląskie,2222
2,2010-12-01,dolnośląskie,2082
3,2010-11-23,dolnośląskie,2230
4,2010-11-16,dolnośląskie,1795


In [4]:
df_t = pd.read_csv("trends.csv", header=0, names=["Token", "Place", "Date", "Value"], parse_dates=['Date'])
df_trends_grypa = df_t[df_t.Token=="grypa"]
df_trends_grypa.head()

,Token,Place,Date,Value
0,grypa,śląskie,2010-01-10,699.486201
1,grypa,śląskie,2010-01-17,782.967092
2,grypa,śląskie,2010-01-24,689.975973
3,grypa,śląskie,2010-01-31,773.999028
4,grypa,śląskie,2010-02-07,600.234477


In [5]:
items_t = [i for i in df_trends_grypa.Date]

def func_t(date, woj, items):
    date = pd.to_datetime(date)
    data_trends = min(items, key=lambda x: abs(x - date)) #najbliższa data --> zbierz z niej wartości
    data_trends = str(data_trends)[0:10]
    try:
        value_trends = df_trends_grypa.Value[(df_trends_grypa.Date == data_trends)&
                                         (df_trends_grypa.Place==woj)].values[0]
    except:
        value_trends = 0
    return value_trends

In [6]:
df_m['Trends_grypa'] = df_m.apply(lambda x: func_t(x.Date, x.Place, items_t), axis=1)
df_m.head()

,Date,Place,Ill,Trends_grypa
0,2010-12-16,dolnośląskie,1528,1486.792129
1,2010-12-08,dolnośląskie,2222,471.267077
2,2010-12-01,dolnośląskie,2082,1142.801026
3,2010-11-23,dolnośląskie,2230,694.166073
4,2010-11-16,dolnośląskie,1795,751.129637


In [15]:
df_m.sort_values(by="Date")
df_m.to_csv("Meld_trends.csv", index=False)

df_m = pd.read_csv("Meld_trends.csv", header=0)
df_m.head()

,Date,Place,Ill,Trends_grypa
0,2010-12-16,dolnośląskie,1528,1486.792129
1,2010-12-08,dolnośląskie,2222,471.267077
2,2010-12-01,dolnośląskie,2082,1142.801026
3,2010-11-23,dolnośląskie,2230,694.166073
4,2010-11-16,dolnośląskie,1795,751.129637


In [17]:
df_k = pd.read_csv("Klimat_woj.csv", header=0, parse_dates=['Date'])

In [19]:
def func_k(date, woj, items):
    date = pd.to_datetime(date)
    data_klimat = min(items, key=lambda x: abs(x - date))
    data_klimat = str(data_klimat)[0:10]
    print(data_klimat)
    value_klimat = np.mean(df_k['Średnia dobowa temperatura'][(df_k.Date == data_klimat)&(df_k.Wojewodztwo==woj)])
    return value_klimat

In [ ]:
items_k = [i for i in df_k.Date] #Klimat

df_m['TempDob'] = df_m.apply(lambda x: func_k(x.Date, x.Place, items_k), axis=1)
# df_m.head()

In [341]:
def func_7k(date, woj):
    date = pd.to_datetime(date)
    ranges = [date-datetime.timedelta(i) for i in range(0,7)]
    ranges = [str(ranges[i])[0:10] for i in range(0,7)]
    value_klimat = np.mean(df_k['Średnia dobowa temperatura']
                    [(df_k.Date.isin(ranges))&(df_k.Wojewodztwo==woj)])
#     print(value_klimat)
    return value_klimat


In [342]:
df_m['TempDob_last7'] =  df_m.apply(lambda x: func_7k(x.Date, x.Place), axis=1)

In [344]:
df_m.head()

,Date,Place,Ill,TempDob_last7
0,2010-12-16,dolnośląskie,1528,-5.385714
1,2010-12-08,dolnośląskie,2222,-5.853061
2,2010-12-01,dolnośląskie,2082,-4.779592
3,2010-11-23,dolnośląskie,2230,4.122449
4,2010-11-16,dolnośląskie,1795,8.483673
